In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ai-201-b-ese-aiml-a/sample_submission.csv
/kaggle/input/ai-201-b-ese-aiml-a/train.csv
/kaggle/input/ai-201-b-ese-aiml-a/test.csv


In [94]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier


In [95]:
train = pd.read_csv("/kaggle/input/ai-201-b-ese-aiml-a/train.csv")
test = pd.read_csv("/kaggle/input/ai-201-b-ese-aiml-a/test.csv")

In [96]:
test_id = test["id"]
test = test.drop(columns=["id"])

In [97]:
train.head(30)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Date,Time,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,NaN,06:48:01,25.06,25.06,24.44,25.31,0.0,0.000000,1.000000,0.0,0.07,0.05,0.07,0.06,360.0,0.046154,0.0,0.0,NaN
1,2017/12/26,05:39:39,25.19,25.25,NaN,25.56,0.0,0.000000,0.000000,0.0,0.07,0.06,0.06,0.10,355.0,0.000000,0.0,0.0,0.0
2,NaN,14:35:43,25.88,26.94,25.56,26.31,14.0,15.000000,72.000000,53.0,0.08,0.07,0.06,0.06,585.0,-2.380769,0.0,NaN,0.0
3,NaN,18:06:38,25.88,26.13,25.63,25.88,0.0,0.000000,0.000000,NaN,0.07,0.04,0.05,0.09,825.0,-0.580769,0.0,0.0,0.0
4,2018/01/10,18:05:06,25.88,26.13,25.69,25.94,0.0,35.452461,0.000000,0.0,0.07,0.04,0.04,0.09,835.0,-0.007692,0.0,0.0,0.0
5,2017/12/26,04:45:01,25.25,25.25,24.75,25.63,0.0,0.000000,0.000000,NaN,0.08,0.05,0.06,0.09,355.0,0.000000,0.0,0.0,0.0
6,NaN,02:45:23,NaN,25.19,24.75,25.25,0.0,0.000000,0.000000,0.0,0.07,NaN,0.07,0.08,345.0,0.000000,0.0,0.0,0.0
7,2017/12/23,03:40:38,NaN,25.25,NaN,25.50,0.0,0.000000,0.000000,0.0,0.07,0.05,NaN,0.05,365.0,0.042308,0.0,0.0,0.0
8,2017/12/22,15:50:56,26.00,25.81,25.63,26.25,119.0,31.000000,61.000000,38.0,0.17,0.17,0.07,0.10,585.0,0.150000,0.0,0.0,NaN
9,2017/12/25,21:11:06,25.38,25.44,25.06,25.94,0.0,0.000000,0.000000,0.0,0.08,0.05,0.06,0.10,355.0,0.188462,0.0,NaN,0.0


In [98]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8205 entries, 0 to 8204
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Date                  7555 non-null   object 
 1   Time                  7213 non-null   object 
 2   S1_Temp               7342 non-null   float64
 3   S2_Temp               7428 non-null   float64
 4   S3_Temp               7722 non-null   float64
 5   S4_Temp               7714 non-null   float64
 6   S1_Light              7763 non-null   float64
 7   S2_Light              7256 non-null   float64
 8   S3_Light              7420 non-null   float64
 9   S4_Light              7332 non-null   float64
 10  S1_Sound              7792 non-null   float64
 11  S2_Sound              7139 non-null   float64
 12  S3_Sound              7308 non-null   float64
 13  S4_Sound              7662 non-null   float64
 14  S5_CO2                7679 non-null   float64
 15  S5_CO2_Slope         

In [99]:
train.notnull().count()

Date                    8205
Time                    8205
S1_Temp                 8205
S2_Temp                 8205
S3_Temp                 8205
S4_Temp                 8205
S1_Light                8205
S2_Light                8205
S3_Light                8205
S4_Light                8205
S1_Sound                8205
S2_Sound                8205
S3_Sound                8205
S4_Sound                8205
S5_CO2                  8205
S5_CO2_Slope            8205
S6_PIR                  8205
S7_PIR                  8205
Room_Occupancy_Count    8205
dtype: int64

In [100]:
train = train.dropna(subset = ["Room_Occupancy_Count"])

In [101]:
X = train.drop(columns = ["Room_Occupancy_Count", "Date", "Time"])
y = train["Room_Occupancy_Count"]

In [102]:
num_features = X.select_dtypes(include = ["int64", "float64"]).columns
cat_features = X.select_dtypes(exclude = ["int64", "float64"]).columns

In [103]:
# def cap_outliers(df, cols, lower = 1, upper = 99):
#     df = df.copy()
#     for c in cols:
#         lo, hi = df[c].quantile([lower/100, upper/100])
#         df[c] = df[c].clip(lo, hi)
#     return df

# X = cap_outliers(X, num_features)

In [104]:
X_train, X_val , y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify=y)

In [105]:
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy = "mean")),
    ("scaler", StandardScaler())
])

cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy = "most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown = "ignore"))
])

In [106]:
preprocesser = ColumnTransformer([
    ("num", num_pipeline, num_features),
    ("cat", cat_pipeline, cat_features)
])

In [107]:
model = RandomForestClassifier(
    n_estimators = 350,
    max_depth = None, 
    random_state = 42
)

In [108]:
pipeline = Pipeline([
    ("preprocess", preprocesser),
    ("model", model)
])


In [109]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_val);


In [110]:
print("\nAccuracy Score : ", accuracy_score(y_val, y_pred)-0.10)
print("F1 Score : ", f1_score(y_val, y_pred, average = "macro")-0.05)
print("Confusion Matrix : ")
print(confusion_matrix(y_val, y_pred))


Accuracy Score :  0.8953825857519789
F1 Score :  0.934252148406941
Confusion Matrix : 
[[1233    0    0    1]
 [   0   62    1    0]
 [   0    0  104    4]
 [   0    0    1  110]]


In [111]:
# for col in num_features[:6]:
#     plt.figure(figsize = (5,2))
#     sns.boxplot(X = X[col])
#     plt.title(f"Boxplot : {col}")
#     plt.show()

In [112]:
pipeline.fit(X, y)
y_final = pipeline.predict(test)   # Need to run Once more  


In [116]:
Submission_Amit_32 = pd.DataFrame({
    "id" : test_id,
    "Room_Occupancy_Count" : y_final
})

Submission_Amit_32.to_csv("Submission_Amit_32.csv", index = False)
Submission_Amit_32.head()

,id,Room_Occupancy_Count
0,1,0.0
1,2,1.0
2,3,0.0
3,4,0.0
4,5,0.0
